In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.6 MB/s eta 0:00:00:00:0100:01
  Obtaining dependency information for wandb from https://files.pythonhosted.org/packages/e1/13/44dda105177622788af8f5da6f9358ecd6fa46e80caa3f4a01ba02cf63d3/wandb-0.15.7-py3-none-any.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/00/45/ec3407adf6f6b5bf867a4462b2b0af27597a26bd3cd6e2534cb6ab029938/filelock-3.12.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00:00:0100:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/11/10/943cfb579f1a02909ff96464c69893b1d25be3731b5d3652c2e0cf1281ea/numpy-1.24.4-cp38-cp38-macosx_10_

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def get_predictions(model, inputs, optimizer):
    optimizer.zero_grad()
    return model(inputs)

def update_model(data, model, criterion, optimizer):
    inputs, labels = data
    preds = get_predictions(model, inputs, optimizer)
    loss = criterion(preds, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def get_transforms(norm=0.5):
    return transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((norm, norm, norm), (norm, norm, norm))])

def get_data(transforms, batch_size=4):
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transforms)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
    return trainloader

def save_model(model, path):
    torch.save(model.state_dict(), path)

In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zain-raza (atlas-applied-analytics). Use `wandb login --relogin` to force relogin


True

In [11]:
def train():
    config = {
        'norm': 0.5,
        'batch_size': 2,
        'lr': 0.001,
        'momentum': 0.9,
        'epochs': 2,
    }
    
    # setup training
    # with wandb.init(project='GPT-5', config=config,
    #                 save_code=True, name='Zain\'s Trying Out .log_code()'):
    #     
    run = wandb.init(project='GPT-5', config=config,
                     save_code=True, name='Zain\'s Trying Out .log_code() 2')
    run.log_code(".", include_fn=lambda path: path.endswith(".ipynb"))

    config = wandb.config

    transforms = get_transforms(config.norm)
    data = get_data(transforms, config.batch_size)
    model = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)
    
    # train model
    for epoch in range(config.epochs): 
        for i, batch in enumerate(data, 0):
            loss = update_model(batch, model, criterion, optimizer)
            
            # log results
            wandb.log({'epoch': epoch, 'loss': loss})
    path = './cifar_net.pth'
    save_model(model, path)
    wandb.finish()

train()

Files already downloaded and verified


KeyboardInterrupt: 

In [3]:
wandb.finish()